IMPORTAÇÃO DE MODULOS

In [39]:
from path_arquivos import *
import pandas as pd
import numpy as np
import glob
import os
funcionario = pd.read_excel(ar_xlsx.ar_func, sheet_name='FUNC')
cons28 = pd.read_excel(bi_abst.cons_28)
cons64 = pd.read_excel(bi_abst.cons_64)


EXTRAÇÃO DAS BASES

In [54]:
func = funcionario[['ID_NOME', 'NOME', 'AREA', 'NAME', 'SETOR']].copy()
func = func.loc[func['AREA'] == 'EXPEDICAO']
grop28 = cons28.groupby(['DATA','CODFUNCOS','FUNCOSFIM']).agg(
    QTDE_OS = ("NUMOS", 'nunique'),
    OS_50 = ("Tipo O.S.", lambda x: (x == '50 - Movimentação De Para').sum()),
    OS_61 = ("Tipo O.S.", lambda x: (x == '61 - Movimentação De Para Horizontal').sum()),
    OS_58 = ("Tipo O.S.", lambda x: (x == '58 - Transferencia de Para Vertical').sum())
).reset_index()

grop64 = cons64.groupby(['DATAGERACAO','CODEMPILHADOR']).agg(
            OS_RECEB = ("NUMOS", 'nunique')
        ).reset_index()

df_os_geral = func.merge(grop28, left_on='ID_NOME', right_on='CODFUNCOS', how= 'left')
df_os_geral = df_os_geral.merge(grop64, left_on=['ID_NOME', 'DATA'], right_on= ['CODEMPILHADOR','DATAGERACAO'], how='left').drop(columns=['DATAGERACAO','CODEMPILHADOR', 'CODFUNCOS', 'FUNCOSFIM'])

df_os_geral = df_os_geral.fillna(0)
df_os_geral = df_os_geral.loc[df_os_geral['DATA'] != 0]
df_os_geral['DATA'] = pd.to_datetime(df_os_geral['DATA']).dt.normalize()
df_os_geral['MES'] = df_os_geral['DATA'].dt.month
df_os_geral = df_os_geral.loc[df_os_geral['MES'] == 8]


In [58]:
display(df_os_geral.columns)
pd.set_option('display.max_rows', None)
display(df_os_geral.loc[df_os_geral['DATA'] == '2025-08-01'].sort_values(by='NOME', ascending= True))




Index(['ID_NOME', 'NOME', 'AREA', 'NAME', 'SETOR', 'DATA', 'QTDE_OS', 'OS_50',
       'OS_61', 'OS_58', 'OS_RECEB', 'MES'],
      dtype='object')

,ID_NOME,NOME,AREA,NAME,SETOR,DATA,QTDE_OS,OS_50,OS_61,OS_58,OS_RECEB,MES
81,180775,ADAEL RAMOS SILVA PAIVA,EXPEDICAO,ADAEL PAIVA,EMPILHADOR,2025-08-01,229.0,97.0,0.0,132.0,0.0,8
251,122268,BRUNO OLIVEIRA SOUSA CARNEIRO,EXPEDICAO,BRUNO CARNEIRO,EMPILHADOR,2025-08-01,367.0,113.0,38.0,216.0,0.0,8
309,180026,BRUNO SILVA GOMES,EXPEDICAO,BRUNOGOMES,FUNC,2025-08-01,111.0,48.0,52.0,11.0,0.0,8
323,180275,CARLIENE DOURADO RODRIGUES,EXPEDICAO,C RODRIGUES,MESA,2025-08-01,1.0,1.0,0.0,0.0,0.0,8
453,122407,CEZAR LUIZ DE SOUZA,EXPEDICAO,CEZAR SOUZA,EMPILHADOR,2025-08-01,15.0,3.0,0.0,12.0,107.0,8
540,180143,CLEBER VIEIRA DA SILVA,EXPEDICAO,CLEBER SILVA,FUNC,2025-08-01,58.0,58.0,0.0,0.0,0.0,8
682,122529,DIONE OLIVEIRA DOS SANTOS,EXPEDICAO,DIONE SANTOS,EMPILHADOR,2025-08-01,262.0,64.0,29.0,169.0,8.0,8
777,180080,ELEILSON FERNANDES DA SILVA,EXPEDICAO,ELEILSON SILVA,FUNC,2025-08-01,2.0,1.0,0.0,1.0,2.0,8
861,1224,ELIEL NUNES DE MORAES,EXPEDICAO,ELIEL,EMPILHADOR,2025-08-01,126.0,9.0,0.0,117.0,19.0,8
930,100846,ELIEL TOMAS GOMES,EXPEDICAO,ELIEL GOMES,FUNC,2025-08-01,2.0,2.0,0.0,0.0,0.0,8
